<a href="https://colab.research.google.com/github/luizvalle/Chessformer/blob/main/chessformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Mount drive {display-mode: "form"}
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# @title Install dependencies {display-mode: "form"}
%%shell
pip install dask

In [3]:
# @title Imports {display-mode: "form"}
import dask.dataframe as dd

## Data analysis

In [ ]:
data = dd.read_parquet(
    path="/content/drive/MyDrive/Trabalhos Escolares/Data Mining/Dataset/Data exploration/*.parquet.zstd",
    compression="zstd"
)
data.head(10)